In [2]:
import torch
import pickle
from server import server, app, decode_img, list_to_numpy
from serverClass import ServerClass
import cv2
import matplotlib.pyplot as plt
import face_recognition

import numpy as np

In [3]:
with open('test_data.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [4]:
server = ServerClass()

imgsL = []
imgsR = []
cam = list_to_numpy(data[0]['cam'])

for obj in data:
  imgL = obj['imgL']
  imgR = obj['imgR'] 
  #cam = obj['cam']

  imgL = decode_img(imgL)
  imgR = decode_img(imgR)

  imgsL.append(imgL)
  imgsR.append(imgR)

In [5]:
imgsL_rect, imgsR_rect = [], []
for imgL, imgR in zip(imgsL, imgsR):
  imgL_rect, imgR_rect = server.rectify_frames(imgL, imgR, cam)
  imgsL_rect.append(imgL_rect)
  imgsR_rect.append(imgR_rect)

#imgL_rect, imgR_rect = server.rectify_frames(imgL, imgR, cam)

rectsL_list=[]
for imgL_rect in imgsL_rect:
  rectsL_list.append(face_recognition.face_locations(imgL_rect))

In [6]:


disparity_maps = []
for imgL_rect, imgR_rect in zip(imgsL_rect, imgsR_rect):
    #for top, right, bottom, left in rects:
        #cv2.rectangle(imgL_rect, (left, top), (right, bottom), (0, 0, 255), 2)
    disparity_map = server.get_disparity(
        img1_rectified=imgL_rect, 
        img2_rectified=imgR_rect, 
        min_disp=-100, 
        max_disp=100, 
        block_size=1, 
        uniquenessRatio=0, 
        speckleWindowSize=0, 
        speckleRange=1, 
        disp12MaxDiff=0
    )
    disparity_maps.append(disparity_map)

In [7]:
"""
for testing get_disparity parameters 


index = 25
test_map = server.get_disparity(
        img1_rectified=imgsL_rect[index], 
        img2_rectified=imgsR_rect[index], 
        min_disp=-100, 
        max_disp=100, 
        block_size=1, 
        uniquenessRatio=0, 
        speckleWindowSize=0, 
        speckleRange=1, 
        disp12MaxDiff=0
    )

for disparity_map in disparity_maps:
    plt.imshow(disparity_map, cmap='gray')
    plt.title('Disparity Map')
    plt.colorbar()
    plt.show()

plt.imshow(disparity_map, cmap='gray')
plt.title('Disparity Map')
plt.colorbar()
plt.show()

cv2.imshow('img1', imgL)
cv2.waitKey(0)
cv2.destroyAllWindows()
"""

"\nfor testing get_disparity parameters \n\n\nindex = 25\ntest_map = server.get_disparity(\n        img1_rectified=imgsL_rect[index], \n        img2_rectified=imgsR_rect[index], \n        min_disp=-100, \n        max_disp=100, \n        block_size=1, \n        uniquenessRatio=0, \n        speckleWindowSize=0, \n        speckleRange=1, \n        disp12MaxDiff=0\n    )\n\nfor disparity_map in disparity_maps:\n    plt.imshow(disparity_map, cmap='gray')\n    plt.title('Disparity Map')\n    plt.colorbar()\n    plt.show()\n\nplt.imshow(disparity_map, cmap='gray')\nplt.title('Disparity Map')\nplt.colorbar()\nplt.show()\n\ncv2.imshow('img1', imgL)\ncv2.waitKey(0)\ncv2.destroyAllWindows()\n"

In [92]:
disparity_faces = []
padding_percentage = 0.20
target_size = (224,224)

print(len(rectsL_list[20]))
print(len(disparity_maps))

for i, rectsL in enumerate(rectsL_list):
    height, width = disparity_maps[i].shape[:2]  # Get the dimensions of the disparity map
    face_region=None

    for x, (top, right, bottom, left) in enumerate(rectsL):

        # Calculate dynamic padding based on the size of the face
        pad_w = int((right - left) * padding_percentage)
        pad_h = int((bottom - top) * padding_percentage)


        
        # Apply padding and ensure indices are within the image bounds
        padded_top = max(top - pad_h, 0)
        padded_bottom = min(bottom + pad_h, height)
        padded_left = max(left - pad_w, 0)
        padded_right = min(right + pad_w, width)
        
        # Extract the face region with padding
        face_region = disparity_maps[i][padded_top:padded_bottom, padded_left:padded_right]
        face_normal = imgsL_rect[i][padded_top:padded_bottom, padded_left:padded_right]

        face_region_resized = cv2.resize(face_region, target_size, interpolation=cv2.INTER_AREA)
        face_normal_resized = cv2.resize(face_normal, target_size, interpolation=cv2.INTER_AREA)

        
        """
        if i == 20:
            cv2.imshow('face', imgsL_rect[i])
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        """

        print(i)

    disparity_faces.append(face_region_resized)


1
40
0
2
3
4
5
6
8
9
10
11
12
13
14
15
16
17
18
20
21
22
23
24
25
26
29
31
32
34
35
36
37
39


In [9]:
"""
for i, (top, right, bottom, left) in enumerate(rectsL):
    # Draw rectangle on the disparity map
    cv2.rectangle(disparity_map, (left, top), (right, bottom), (255, 0, 0), 2)

cv2.imshow('Disparity Map with Face Regions', disparity_map)
cv2.waitKey(0)  # Wait for a key press to close
cv2.destroyAllWindows()
"""

"\nfor i, (top, right, bottom, left) in enumerate(rectsL):\n    # Draw rectangle on the disparity map\n    cv2.rectangle(disparity_map, (left, top), (right, bottom), (255, 0, 0), 2)\n\ncv2.imshow('Disparity Map with Face Regions', disparity_map)\ncv2.waitKey(0)  # Wait for a key press to close\ncv2.destroyAllWindows()\n"

In [10]:
# Display the disparity map using matplotlib

"""
for face in disparity_faces:
  plt.figure(figsize=(10, 7))
  plt.imshow(face, cmap='gray')
  plt.title('Disparity Map')
  plt.colorbar()  # Adds a colorbar to interpret the values
  plt.show()
"""

"\nfor face in disparity_faces:\n  plt.figure(figsize=(10, 7))\n  plt.imshow(face, cmap='gray')\n  plt.title('Disparity Map')\n  plt.colorbar()  # Adds a colorbar to interpret the values\n  plt.show()\n"

In [124]:
index = 21
plt.figure(figsize=(10, 7))
plt.imshow(disparity_maps[index], cmap='gray')
plt.title('Disparity Map')
plt.colorbar()  # Adds a colorbar to interpret the values
plt.show()

In [129]:
X_train = []
y_train = []
for i, face in enumerate(disparity_faces):
  if face is not None:
    X_train.append(face)

    #1 for valid face, 0 for invalid face
    if i > 21:
      y_train.append(0)
    else:
      y_train.append(1)

In [130]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.act3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool1(self.act1(self.conv1(x)))
        x = self.pool2(self.act2(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = self.act3(self.fc1(x))
        x = self.fc2(x)
        return x

In [164]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
print(X_train_tensor.shape)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True)

torch.Size([40, 1, 224, 224])


In [139]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Epoch 1, Loss: 55.80411946773529
Epoch 2, Loss: 92.74967041015626
Epoch 3, Loss: 16.947527122497558
Epoch 4, Loss: 0.0
Epoch 5, Loss: 2.0714488983154298
Epoch 6, Loss: 0.0
Epoch 7, Loss: 0.0
Epoch 8, Loss: 0.0
Epoch 9, Loss: 0.0
Epoch 10, Loss: 0.0


In [163]:
# set model to eval mode
model.eval()

#prep input image
test_image = torch.tensor(X_train[22], dtype=torch.float32).unsqueeze(0)
test_image = test_image.unsqueeze(1)
test_image = test_image.to(device) 
print(test_image.shape)

with torch.no_grad(): 
    output = model(test_image)

probabilities = torch.nn.functional.softmax(output, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)

print(f"Predicted class: {predicted_class.item()}")
print(f"Probabilities: {probabilities}")

torch.Size([1, 1, 224, 224])
Predicted class: 0
Probabilities: tensor([[1., 0.]], device='cuda:0')


In [ ]:
torch.save(model.state_dict(), 'model_state_dict.pth')